In [9]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import cell2location
import scvi
import torch
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.backends.backend_pdf as mpdf
import warnings
sys.path.append("/home/biolab/Projects/LAMs_wd/scripts")
warnings.filterwarnings('ignore')
data_type = 'float32'
from visium_qc_and_visualisation import read_and_qc
from config import lung_config

In [10]:
env = "Tumour"

Load the signature matrix

In [11]:
o_sig_matrix = pd.read_csv("/home/biolab/Projects/LAMs_wd/data/lung_adenocarcinoma_cibersortx_macs_sig_mtrx_renamed.csv", index_col=0)
o_sig_matrix

,RTM_TAMs,Prolif_TAMs,LA_TAMs,Inflam_TAMs,Reg_TAMs,Angio_TAMs,IFN_TAMs
ZCCHC10,1.000000,104.665882,1.000000,1.0,1.000000,1.000000,72.191737
SGPP1,1.000000,1.000000,1.000000,1.0,77.811270,1.000000,1.000000
KDM5A,56.987988,1.000000,79.586648,1.0,1.000000,1.000000,1.000000
HLA-DQB2,69.916245,1.000000,1.000000,1.0,146.979570,1.000000,1.000000
ANKH,1.000000,1.000000,66.086644,1.0,60.568766,1.000000,1.000000
...,...,...,...,...,...,...,...
SKIC3,1.000000,1.000000,1.000000,1.0,1.000000,71.534528,1.000000
MRPL24,66.116234,115.637194,1.000000,1.0,68.622728,1.000000,71.890028
ROGDI,116.705054,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
LTC4S,1.000000,1.000000,1.000000,1.0,74.910003,1.000000,1.000000


In [ ]:
with open(f"/home/biolab/Projects/LAMs_wd/luad_macrophage_spots_cell2location_spatial_model_SC_Y_sig_matrix_adeno_LUSC_patients_{env}.txt", "r") as i:
    spots = i.read().split("\n")

In [5]:
len(spots)

497

In [6]:
spots[:10]

['spaceranger110_count_36210_OTAR_LNGsp9476042_GRCh38-2020-A_ACTGTAGCACTTTGGA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476043_GRCh38-2020-A_TCGTGTCACGCTGACA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476042_GRCh38-2020-A_TCGGAGAGTATCGGGA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476042_GRCh38-2020-A_TTCTAACCGAAGCTTA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476045_GRCh38-2020-A_CCATTTCTACCTATTA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476045_GRCh38-2020-A_CCCAGTTAAGGCGCCG-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476045_GRCh38-2020-A_TAAACCCAGGAGGGCA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476043_GRCh38-2020-A_ACCGACACATCTCCCA-1',
 'spaceranger110_count_36210_OTAR_LNGsp9476045_GRCh38-2020-A_GCGGTTCCCTATCATG-1',
 'spaceranger110_count_38262_OTAR_LNGsp10206167_GRCh38-2020-A_GCGTCTCTGCATTGGG-1']

Run Cell2location

In [ ]:
def run_cell2loc(ifolder, environment):
    seed = 42
    torch.manual_seed(seed)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    outDir = f"/home/biolab/Projects/LAMs_wd/results/cell2location_task2/"
    regDir = f"/home/biolab/Projects/LAMs_wd/data/E-MTAB-13530/{ifolder}/cell2location/regression_model/"
 

    inf_aver = None
    # Export estimated expression signature in each cluster
    
    inf_aver = o_sig_matrix.copy()

    # Scale up by average sample scaling factor. This corrects for sequencing depth
    # inf_aver = inf_aver * adata_sc_model.uns["mod"]["post_sample_means"]["sample_scaling"].mean() # Need to check if this is still necessary

    # Pick up Visium
    if env == "Tumour":
        sample_names = lung_config[environment]["spatial"]["LUSC"]
    else:
        sample_names = lung_config[environment]["spatial"]
    print(sample_names)
    
    ####
    #assign you path here, where your patients' data are located
    ############################
    inDir = "/home/biolab/Projects/LAMs_wd/data/E-TAMB-13530/"
    ########################################################################
    
    slides = [read_and_qc(sample_name, path=inDir) for sample_name in sample_names]
    slides_ = []
    
    #now here i select a subset of spots
    for slide in slides:
        slides_.append(slide[list(set(slide.obs_names).intersection(spots))])
    slides = slides_
    
    print(slides)
    
    
    adata_vis = slides[0].concatenate(slides[1:], batch_key="sample", uns_merge="unique", batch_categories=sample_names, index_unique=None)

    # Mitochondria-encoded (MT) genes should be removed for spatial mapping
    # Find mitochondria-encoded (MT) genes
    adata_vis.var["MT_gene"] = [gene.startswith("MT-") for gene in adata_vis.var["SYMBOL"]]
    # Remove MT genes for spatial mapping (keeping their counts in the object)
    adata_vis.obsm["MT"] = adata_vis[:, adata_vis.var["MT_gene"].values].X.toarray()
    adata_vis = adata_vis[:, ~adata_vis.var["MT_gene"].values]
    # Reset the index for compatibility with the scRNA-seq counts
    adata_vis.var.set_index("SYMBOL", inplace=True)

    # Find shared genes and subset both anndata and reference signatures
    adata_vis.var_names_make_unique()
    
    inf_aver_index = np.array(inf_aver.index, dtype=str)
    adata_vis_var_names = np.array(adata_vis.var_names, dtype=str)
    intersect = np.intersect1d(adata_vis_var_names, inf_aver_index)
    #intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
    #adata_vis = adata_vis[:, intersect] # This doesn't work -- think there's some incompatibility between ScanPy and AnnData versions. Use the line below instead
    adata_vis = adata_vis[:, adata_vis.var.index.isin(intersect)].copy()
    inf_aver = inf_aver.loc[intersect, :]
    # Reindex for cell2location
    inf_aver = inf_aver.reindex(adata_vis.var.index)


    # Training cell2location
    cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")
    mod = cell2location.models.Cell2location(adata_vis,
                                             cell_state_df=inf_aver,
                                             detection_alpha=200, # Controls the normalisation of the within-experiment variation of the RNA detection (default value for now)
                                             N_cells_per_location=8
                                             )
    '''
                                             **model_kwargs={
                                                # Prior on the number of cells, cell types and co-located groups. Hyperparameter inputs go here
                                                "cell_number_prior": {
                                                    # - N - the expected number of cells per location:
                                                    "cells_per_spot": 8,
                                                    # - A - the expected number of cell types per location:
                                                    "factors_per_spot": 9,
                                                    # - Y - the expected number of co-located cell type groups per location
                                                    "combs_per_spot": 5
                                                },
                                                # Prior beliefs on the sensitivity of spatial technology:
                                                "gene_level_prior": {
                                                    # Prior on the mean
                                                    "mean": 0.5,
                                                    # Prior on standard deviation
                                                    # A good choice of this value should be at least 2 times lower that the mean
                                                    "sd": 0.15
                                                }
                                             }
                                             )
    '''

    # Try 2500 batches initially, rather than training on the entire data-set
    mod.train(batch_size=2500, train_size=1, max_epochs=3000)
    mod.save(f"{outDir}{environment}", overwrite=True)
    adata_vis = mod.export_posterior(adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': 2500})
    adata_vis.write(f"{outDir}{environment}/sp_our_sig_matrix_macrophage_cts_{environment}.h5ad")

    train_performance = mod.history["elbo_train"]
    train_performance.plot(logy=True)
    plt.xlabel("Training epochs")
    plt.ylabel("-ELBO loss")
    plt.title(f"Lung ({environment})")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(f"{outDir}{environment}/ELBO_Cell2location_{environment}.png")
    plt.close()
    

In [8]:
run_cell2loc("dataset", env)

['spaceranger110_count_36210_OTAR_LNGsp9476045_GRCh38-2020-A', 'spaceranger110_count_38262_OTAR_LNGsp10206168_GRCh38-2020-A', 'spaceranger110_count_38262_OTAR_LNGsp10206167_GRCh38-2020-A', 'spaceranger110_count_36210_OTAR_LNGsp9476042_GRCh38-2020-A', 'spaceranger110_count_36210_OTAR_LNGsp9476043_GRCh38-2020-A']
[View of AnnData object with n_obs × n_vars = 116 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac'
    var: 'feature_types', 'genome', 'SYMBOL', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mt'
    uns: 'spatial'
    obsm: 'spatial', View of AnnData object with n_obs × n_vars = 78 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 3000/3000: 100%|██████████| 3000/3000 [03:56<00:00, 12.57it/s, v_num=1, elbo_train=2.59e+5]

`Trainer.fit` stopped: `max_epochs=3000` reached.


Sampling global variables, sample: 100%|██████████| 999/999 [00:13<00:00, 71.74it/s]
